In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm


In [2]:
S0=1282.979126
sigma=0.2974196854602998

In [3]:
r = 0.06          # 6% annual risk-free rate (India)
T = 0.5           # 6 months to maturity

In [4]:
K = S0            # Strike price

In [5]:
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    call = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    return call

def black_scholes_put(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    put = K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put


In [6]:
bs_call = black_scholes_call(S0, K, T, r, sigma)
bs_put  = black_scholes_put(S0, K, T, r, sigma)

bs_call, bs_put

(np.float64(125.88688620062669), np.float64(87.96912266329537))

In [7]:
N = 100           # number of time steps
dt = T / N

u = np.exp(sigma * np.sqrt(dt))
d = 1 / u

p = (np.exp(r * dt) - d) / (u - d)

In [8]:
stock_tree = np.zeros((N+1, N+1))

for i in range(N+1):
    for j in range(i+1):
        stock_tree[j, i] = S0 * (u**(i-j)) * (d**j)


In [9]:
option_tree = np.maximum(stock_tree[:, N] - K, 0)

In [10]:
option_tree

array([9226.31037373, 8793.44131913, 8378.40178749, 7980.45739532,
       7598.90400778, 7233.06649276, 6882.29752632, 6545.97644723,
       6223.50815881, 5914.32207593, 5617.87111537, 5333.63072786,
       5061.09796985, 4799.79061365, 4549.24629411, 4309.02169055,
       4078.69174228, 3857.84889655, 3646.10238732, 3443.07754395,
       3248.41512811, 3061.77069827, 2882.81400011, 2711.22838224,
       2546.71023589, 2388.96845767, 2237.72393452, 2092.7090498 ,
       1953.66720978, 1820.35238965, 1692.52869811, 1569.96996007,
       1452.45931639, 1339.78884019, 1231.75916892, 1128.17915161,
       1028.86551065,  933.64251749,  842.34168166,  754.80145273,
        670.86693436,  590.38961027,  513.22708147,  439.24281422,
        368.30589854,  300.29081651,  235.07722018,  172.54971864,
        112.59767383,   55.11500481,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.     

In [11]:
for i in range(N-1, -1, -1):
    for j in range(i+1):
        option_tree[j] = np.exp(-r*dt) * (
            p * option_tree[j] + (1-p) * option_tree[j+1]
        )

binomial_call = option_tree[0]
binomial_call


np.float64(125.61998228750188)

In [12]:
comparison = pd.DataFrame({
    "Model": ["Black-Scholes", "Binomial Tree"],
    "Call Price": [bs_call, binomial_call]
})

comparison


,Model,Call Price
0,Black-Scholes,125.886886
1,Binomial Tree,125.619982
